In [28]:
import numpy as np
import pandas as pd
from scipy import stats

In [50]:
def percentile( n ):
    def percentile_( x ):
        return np.percentile( x, n )
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [51]:
meta = pd.read_csv( './data/meta/datos_10replicas.csv' )

In [52]:
del meta[ 'Unnamed: 0' ]

In [53]:
meta.shape

(1070, 11)

In [54]:
meta.dtypes

replicate      int64
time           int64
S            float64
E            float64
A            float64
I1           float64
I2           float64
I3           float64
R            float64
D            float64
obs          float64
dtype: object

In [55]:
meta.tail()

,replicate,time,S,E,A,I1,I2,I3,R,D,obs
1065,10,102,7.129246e+06,1944.679581,1779.588349,765.347201,632.858271,257.234520,14467.258397,498.130817,460.0
1066,10,103,7.128829e+06,1969.847524,1802.567058,775.229650,641.730681,261.637901,14793.264194,511.102702,501.0
1067,10,104,7.128407e+06,1995.337897,1825.849823,785.242863,650.657938,266.025543,15123.592158,524.294345,579.0
1068,10,105,7.127980e+06,2021.153989,1849.439217,795.387949,659.646085,270.401569,15458.290438,537.705063,526.0
1069,10,106,7.127547e+06,2047.299163,1873.337915,805.666057,668.700742,274.769975,15797.408287,551.334375,546.0


In [56]:
meta.groupby( 'time' ).agg( { 'obs': [ 'min', 'mean', 'median', 'max', percentile( 2.5 ), percentile( 25 ), percentile( 75 ), percentile( 97.5 ) ] } ).tail()

obs                                                                  \
        min   mean median    max percentile_2.5 percentile_25 percentile_75   
time                                                                          
102   460.0  497.0  497.0  533.0        463.375        482.25        504.00   
103   486.0  508.0  505.5  543.0        487.800        498.75        509.75   
104   485.0  526.1  521.0  579.0        490.625        513.50        534.75   
105   489.0  529.9  529.0  562.0        493.950        524.25        543.50   
106   518.0  543.8  544.0  581.0        519.350        526.75        550.00   

                      
     percentile_97.5  
time                  
102          532.100  
103          539.625  
104          572.925  
105          558.400  
106          578.525

In [95]:
to_export = meta.loc[ meta[ 'replicate' ] == 1 ][ [ 'time', 'I2', 'I3', 'D' ] ].round( 0 )

In [96]:
to_export[ 'I2' ] = to_export[ 'I2' ].astype( int )
to_export[ 'I3' ] = to_export[ 'I3' ].astype( int )
to_export[ 'D' ] = to_export[ 'D' ].astype( int )

In [97]:
to_export.head()

,time,I2,I3,D
0,0,0,0,0
1,1,0,0,0
2,2,0,0,0
3,3,0,0,0
4,4,0,0,0


In [98]:
to_export = to_export.iloc[ 1: ]

In [99]:
to_export.tail()

,time,I2,I3,D
102,102,633,257,498
103,103,642,262,511
104,104,651,266,524
105,105,660,270,538
106,106,669,275,551


In [100]:
real = pd.read_csv( './data/meta/BOG17jun.csv' )

In [101]:
real.shape

(96, 3)

In [102]:
real[ 'date' ] = pd.to_datetime( real[ 'date' ], format = '%d/%m/%Y' )

In [103]:
real.dtypes

date    datetime64[ns]
time             int64
obs              int64
dtype: object

In [104]:
real.head()

,date,time,obs
0,2020-03-14,1,0
1,2020-03-15,2,0
2,2020-03-16,3,0
3,2020-03-17,4,0
4,2020-03-18,5,0


In [105]:
to_export = real.merge( to_export, how = 'outer', on = 'time' )

In [106]:
to_export.rename( columns = { 'date': 'Fecha', 'obs': 'Fallecidos_reales', 'D': 'Fallecidos', 'I2': 'Graves', 'I3': 'Críticos' }, inplace = True )

In [107]:
def impute_date( x ):
    if pd.isnull( x[ 'Fecha' ] ):
        return max( to_export[ 'Fecha' ] ) + pd.DateOffset( x[ 'time' ] - 96 )
    else:
        return x[ 'Fecha' ]
    
to_export[ 'Fecha' ] = to_export.apply( lambda x: impute_date( x ), axis = 1 )

In [108]:
to_export.head( 25 )

,Fecha,time,Fallecidos_reales,Graves,Críticos,Fallecidos
0,2020-03-14,1,0.0,0,0,0
1,2020-03-15,2,0.0,0,0,0
2,2020-03-16,3,0.0,0,0,0
3,2020-03-17,4,0.0,0,0,0
4,2020-03-18,5,0.0,0,0,0
5,2020-03-19,6,0.0,0,0,0
6,2020-03-20,7,0.0,0,0,0
7,2020-03-21,8,0.0,1,0,0
8,2020-03-22,9,0.0,1,0,0
9,2020-03-23,10,0.0,2,0,0


In [109]:
to_export = pd.melt( to_export, id_vars=[ 'Fecha' ], value_vars = [ 'Fallecidos_reales', 'Graves', 'Críticos', 'Fallecidos' ] )

In [110]:
#to_export.drop_duplicates( subset = [ 'value' ], inplace = True )

In [111]:
to_export.head()

,Fecha,variable,value
0,2020-03-14,Fallecidos_reales,0.0
1,2020-03-15,Fallecidos_reales,0.0
2,2020-03-16,Fallecidos_reales,0.0
3,2020-03-17,Fallecidos_reales,0.0
4,2020-03-18,Fallecidos_reales,0.0


In [112]:
to_export.to_csv( './agents-viz/data/meta.csv', index = False )